In [6]:
! pip freeze > requirements.txt

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import plotly.express as px
import pandas as pd

## BeautifulSoup setup and testing

In [19]:
html_doc = """<html><head><title>The Dormouse's story</title></head>
<body>
<p class="title"><b>The Dormouse's story</b></p>

<p class="story">Once upon a time there were three little sisters; and their names were
<a href="http://example.com/elsie" class="sister" id="link1">Elsie</a>,
<a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
<a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>

<p class="story">...</p>
"""

In [20]:
soup = BeautifulSoup(html_doc)

In [22]:
soup.title

<title>The Dormouse's story</title>

In [23]:
soup.title.string

"The Dormouse's story"

In [17]:
soup.a

<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>

In [21]:
soup.find_all('a')

[<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>,
 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>,
 <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>]

In [31]:
for link in soup.find_all('a'):
    print(link.get('href'))

http://example.com/elsie
http://example.com/lacie
http://example.com/tillie


In [33]:
print(soup.get_text())

The Dormouse's story

The Dormouse's story
Once upon a time there were three little sisters; and their names were
Elsie,
Lacie and
Tillie;
and they lived at the bottom of a well.
...



In [39]:
soup.find('head')

<head><title>The Dormouse's story</title></head>

## Web scraping

In [ ]:
#URL : https://www.basketball-reference.com/

### LeBron James vs Michael Jordan season-wise points per minute tally

In [20]:
lebron_james_url = 'https://www.basketball-reference.com/players/j/jamesle01.html'
michael_jordan_url = 'https://www.basketball-reference.com/players/j/jordami01.html'

In [21]:
urls = [lebron_james_url, michael_jordan_url]

In [43]:
# Configure Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode

# Set the path to the ChromeDriver executable
chrome_driver_path = "C:/Users/funde/Downloads/chromedriver_win32.zip/chromedriver.exe"

# Initialize the WebDriver
driver = webdriver.Chrome(executable_path=chrome_driver_path, options=chrome_options)

C:\Users\funde\AppData\Local\Temp\ipykernel_19308\593687960.py:9: DeprecationWarning:

executable_path has been deprecated, please pass in a Service object



In [44]:
ages = []
points = []
player = []
for url in urls:
    driver.get(url)

    # Get the page source (HTML)
    html = driver.page_source
    soup = BeautifulSoup(html)
    title_arr = soup.title.string.split(' ')
    try:
        graph_title = ' '.join(title_arr[0 : title_arr.index('Stats')])
    except:
        graph_title = ' '.join(title_arr[0 : 2])
    all_ids = soup.find_all(id=True)
    for ele in all_ids:
        if ele['id'][0 : len('totals.')] == 'totals.':
            # rows.append()
            # for child in ele.findChildren():
            points.append(float(int(ele.find(attrs={"data-stat" : "pts"}).string) / int(ele.find(attrs={"data-stat" : "mp"}).string)))
            ages.append(int(ele.find(attrs={"data-stat" : "age"}).string))
            player.append(graph_title)
                # print(child)
df = pd.DataFrame(dict(
    age = ages,
    points_per_minute = points,
    player=player
))
df = df.sort_values(by="age")
fig = px.line(df, x="age", y="points_per_minute", color='player', markers=True)
fig.show()
# Close the WebDriver
driver.quit()